# Backoff

Truancated exponetial backoff 란 네트워크 장애를 극복하는 방법중 하나이다.  
보통 클라이언트 측면에서 사용하는 방법이다. 네트워크와의 통신에 문제가 발생할 때, 여러번 재시도 하는 것을 기본으로한다.

다음과 같은 상황에서 많이 사용한다.  
* 데이터를 다운 받는데 500번대 에러나 429에러를 발생시킬때  
* 408 response error 발생  
* 소켓 타임아웃 및 tcp 연결장애

이런 경우 매우 중요하다. 
1. 고객측면에서 어플리케이션을 개발(특히 json/XML api를 이용할때)
2. 그외 등등

### Example algorithm
An exponential backoff algorithm retries requests exponentially, increasing the waiting time between retries up to a maximum backoff time. An example is:

1. If the request fails, wait 1 + random_number_milliseconds seconds and retry the request.

2. If the request fails, wait 2 + random_number_milliseconds seconds and retry the request.

3. If the request fails, wait 4 + random_number_milliseconds seconds and retry the request.

4. And so on, up to a maximum_backoff time.

5. Continue waiting and retrying up to some maximum number of retries, but do not increase the wait period between retries.

where:

*  The wait time is min(((2^n)+random_number_milliseconds), maximum_backoff), with n incremented by 1 for each iteration (request).

*  random_number_milliseconds is a random number of milliseconds less than or equal to 1000. This helps to avoid cases where many clients get synchronized by some situation and all retry at once, sending requests in synchronized waves. The value of random_number_milliseconds is recalculated after each retry request.

*  maximum_backoff is typically 32 or 64 seconds. The appropriate value depends on the use case.

### \+ typing 모듈  
typing 모듈은 런타임단계로서 함수형식이나 객체, 타입에 대한 제한을 두지 않는다.  
IDE, LINTER 등에서 돕기도 하지만 typing 모듈을 이용하여 해결할 수 있다. 

In [ ]:
from functools import wraps
import random
from urllib import HttpError
import time
from typing import Callable

class BackOff(object):
    def __init__(self, maxTries: int=3, 
                is_success: Callable:None, 
                method='exponential', sleepTimes=100, 
                jitter: bool=True,
                action_when_error: Callable:None, actionWhenErrorArugmet: dict=None
                ):
        self.maxTries = maxTries
        self.is_success = is_success
        self.method = method
        self.sleepTimes = sleepTimes
        self.action_when_error = action_when_error
        self.actionWhenErrorArugmet = actionWhenErrorArugmet

    def __call__(self, func):
        @wraps(func)
        def retry(*args, **kwargs):
            tries = 0 
            if self.method = "Exponential":
                sleep_times = self.exponential_wait()
            elif self.method = "Simple":
                sleep_times = self.constant_wait(self.maxTries, self.sleepTimes)
            else: 
                print("Available backoff methods are Exponential/Simple.")
                raise AssignError

            while True:
                tries += 1
                # is_success 함수는 우리가 데코레이터로 덧씌울 함수의 결과값을 인자로 받아 이 함수를 끝낼지 말지를 반환한다.
                # 만약 is_success 함수가 없다면, 즉 그냥 함수 output만으로 끝맺음을 결정할 수 있다면..
                if is_success is None:
                    # 함수 다시시도해보기
                    try:
                        output = func(*arg, **kwargs)
                        finish = True
                    # http 에러인 경우와 아닌경우 로 나누자!
                    except HttpError as e:
                        # 404에러는 기다린다고 나아지는게 아님 따라서 404났을때의 행동수칙을 따로 지정
                        if e.resp.status = 404:
                            output = self.action_when_error(**actionWhenErrorArugmet)
                            finish = True
                        # 만약 404 에러가 아니면 다시 시도할 것
                        else:
                            output = None
                            finish = False
                        # 만약 retry 할때 따로 하고싶은 행동이 있다면 지금 수행
                        if self.retry_action is not None:
                            output = self.retry_action()
                    # http 에러가 아닌 다른 에러라면? 
                    except Exception as e:
                        output = None 
                        finish = False
                        if self.retry_action is not None:
                            output = self.retry_action()
                # 만약 is_success 함수가 있는 경우
                else:
                    output = func(*args, **kwargs)
                    finish = self.is_success(output)
                # 성공하든 아니면 404가 뜨든 끝나면 output 반환
                if finish:
                    return output
                # 성공 안하고 404도 아는데 최대시도 횟수를 초과한경우 
                if self.maxTries is not None and tries >= maxTries:
                    raise maxTriesExceeded
                # 휴식시간 지정
                sleep_time = next(sleep_time)
                # 지터온이라면 full jitter 이용 sleep time 만들기
                if self.jitter:
                    sleep_time = self.full_jitter(sleep_time)
                # 휴식 
                time.timesleep(sleep_time)
        return retry
                
            
    def exponential_wait(self, base=2, maxValuse=64):
        count = 0
        while True:
            value = base ** count + random.random()
            if value <= maxValuse:
                yield value
                count += 1
            else :
                maxValuse += random.random()
                yield maxValuse
    
    def constant_wait(self, maxTries=5, sleepTimes=100):
        for _ in range(maxTries):
            yield sleepTimes

    def full_jitter(self, value):
        return random.uniform(0, value)